# Project Initialisation

In [ ]:
# Kedro 
import os
import sys
from pathlib import Path

# Set Kedro project path
project_path = Path.cwd().parent

# Bootstrap Kedro
from kedro.framework.startup import bootstrap_project
from kedro.framework.session import KedroSession

bootstrap_project(project_path)
session = KedroSession.create(project_path=project_path)
context = session.load_context()
catalog = context.catalog

# Add src/ to Python path
sys.path.append(str(project_path / "src"))

In [ ]:
import warnings
warnings.filterwarnings("ignore", category=FutureWarning, module="seaborn")

In [ ]:
# Import full modules (for reload)
import egt305_job_market_analysis.utils.viz as viz

import importlib
importlib.reload(viz)

# Set custom plot style for consistency
viz.set_plot_style()

# Data Injestion

In [ ]:
# Loading Datasets
df_employee = catalog.load("employee_dataset")
df_salary = catalog.load("employee_salaries")

# Data Cleaning

## Employee Dataset

### Initial Inspection

In [ ]:
import pandas as pd
from IPython.display import display

# Inspecting the employee dataset for basic information & statistics

# Shape of the dataset
print(f"Dataset shape: {df_employee.shape}")

# 2. Preview first 5 rows
display(df_employee.head())

# 3. Column names and data types
df_employee.info()

# 4. Descriptive statistics for numerical and categorical features
display(df_employee.describe(include='all'))

# 5. Check for missing values
missing_counts = df_employee.isnull().sum()
missing_perc = (missing_counts / len(df_employee) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_perc})
display(missing_df[missing_df['Missing Count'] > 0])


Initial issues or anomalies detected.

- Column names are not in a standard format as they have upper & lower case
- Data entries are unecessarily complex i.e. COMP37 as they could just be 37
- Columns are not in the correct dtype
- distanceFromCBD has a very large difference from 75% to MAX indicating high value outliers
- missing data in multiple columns

### Cleaning Employee Dataset

In [ ]:
# initial column names from data description
# standardizing column names for consistency
df_employee.rename(columns={
    'jobId': 'job_id',
    'companyId': 'company_id',
    'jobRole': 'job_role',
    'education': 'education',
    'major': 'major',
    'Industry': 'industry',
    'yearsExperience': 'years_experience',
    'distanceFromCBD': 'distance_from_cbd'
}, inplace=True)

df_employee.head(2)

Fixed column names to be more standardized

In [ ]:
# Check value counts for company_id including NaN
company_counts = df_employee['company_id'].value_counts(dropna=False)

display(company_counts)
print(f"Unique company_id count (including NaN): {df_employee['company_id'].nunique(dropna=False)}")

There is a good data spread in the company_id column

Checking the various unique entries as well as ensuring the prefix is COMP for all, as well as keeping a before prefix drop state.

In [ ]:
import pandas as pd

# Remove 'COMP' prefix and convert to integer
df_employee['company_id'] = (
    df_employee['company_id']
    .astype(str)
    .str.replace('COMP', '', regex=False)
    .replace('<NA>', pd.NA)  # make sure string '<NA>' is real missing value
    .astype('Int64')  # nullable integer dtype just for eda purposes
)

In [ ]:
# Check value counts for company_id including NaN
company_counts = df_employee['company_id'].value_counts(dropna=False)

display(company_counts)
print(f"Unique company_id count (including NaN): {df_employee['company_id'].nunique(dropna=False)}")

Fixed company id column to be more model friendly whilst maintaining all entries including NA (to be dealt with when handling missing or dupe data)

Next lets ensure the job_id column follows the same rules

In [ ]:
import re

# pattern check for job_id
pattern_check_job = df_employee['job_id'].apply(
    lambda x: pd.isna(x) or bool(re.match(r'^JOB\d+$', str(x)))
)
print(f"All non-null job_id match 'JOBxxxx...' format?: {pattern_check_job.all()}")

# Remove 'JOB' prefix and convert to nullable integer
df_employee['job_id'] = (
    df_employee['job_id']
    .astype(str)
    .str.replace('JOB', '', regex=False)
    .replace(['<NA>', 'nan', 'NaN'], pd.NA)  # ensure true missing values
    .astype('Int64')  # nullable integer dtype
)

df_employee.head(2)

In [ ]:
# Identify all string columns
string_cols = df_employee.select_dtypes(include='string').columns

# Display unique values for each string column
for col in string_cols:
    temp_series = df_employee[col].astype(str)  # ensure string format for display
    
    print(f"\n--- {col} ---")
    print(f"Unique values: {temp_series.nunique(dropna=False)}")
    print(temp_series.value_counts(dropna=False))

- job_role column seems to have a good spread of job roles except for a sole exception which is the president job role but all roles are valid and do not have semantic overlap. Some missing data but will be handled later.

- education column has a large amount of missing data which is labeled as NONE and NA. Good spread of data without any semantic overlap.

- major column has a similar issue with education column with missing data with 2 different labels but aside from missing data, there is a good spread of data among the categories with no semantic overlap.

- industry column has a similar issue with job_role column. One singular entry in a category i.e. governement. However, there is a good spread of data.

We will treat NA and NONE as the same for those columns that have both appear at the same time. For the industry & job_role column which only have NA, as there is no clear category or large amount of missing rows, we will drop the entire row which has the missing data. For numeric columns as they only have <500 missing values, we will also just drop the entire row as they are not a significant portion of data as seen in initial inspection, this also includes the missing entries in job_id. This no tolerance for missing data will ensure the data is as complete as can be baring those columns which have a large portion of missing data which will be explicitly labeled as NONE.

In [ ]:
# Columns where NA/NaN should be treated as 'NONE'
cols_na_to_none = ['education', 'major']

for col in cols_na_to_none:
    df_employee[col] = (
        df_employee[col]
        .replace(['NA', 'na', 'NaN', 'nan', '<NA>'], pd.NA)  # unify NA forms
        .fillna('NONE')  # replace actual missing with 'NONE'
    )

# Preview changes
for col in cols_na_to_none:
    print(f"\n--- {col} ---")
    print(df_employee[col].value_counts(dropna=False))


In [ ]:
# Drop rows with missing values in job_role, or industry
cols_drop_na = ['job_role', 'industry']

before_drop = len(df_employee)
df_employee = df_employee.dropna(subset=cols_drop_na)
after_drop = len(df_employee)

print(f"Dropped {before_drop - after_drop} rows due to NA in {cols_drop_na}")

In [ ]:
# Drop rows with missing data in integer columns
int_cols = df_employee.select_dtypes(include=['int64', 'Int64']).columns

before_drop = len(df_employee)
df_employee = df_employee.dropna(subset=int_cols)
after_drop = len(df_employee)

print(f"Dropped {before_drop - after_drop} rows due to NA in integer columns: {list(int_cols)}")

In [ ]:
# Check for remaining missing values in all columns
missing_counts = df_employee.isna().sum()
missing_perc = (missing_counts / len(df_employee) * 100).round(2)

missing_df = pd.DataFrame({
    'Missing Count': missing_counts,
    'Missing %': missing_perc
}).sort_values(by='Missing Count', ascending=False)

display(missing_df[missing_df['Missing Count'] > 0])

This means there is no longer any column with NA entry they are either specifically mapped to NONE or dropped.

Next lets fix the datatypes of the string columns

In [ ]:
# Step: Check datatypes & memory usage of all columns
df_employee.info()

- Since we no longer have any nulls, nullable ints are no longer needed. In data preparation pipeline, we will ensure that missing data is dropped immediately and not converted to Int64 and then dropped later which would be less memory efficient (Done here to help with flow of EDA)

- To ensure memory usage is minimal, we will convert string to categorical dtype

- One specific case will be company_id as it is nominal and not ordinal, we will convert it to a categorical as well.

In [ ]:
# Convert company_id to category
if 'company_id' in df_employee.columns:
    df_employee['company_id'] = df_employee['company_id'].astype('category')

# Convert string columns to category for efficient EDA
for col in df_employee.select_dtypes(include='string').columns:
    df_employee[col] = df_employee[col].astype('category')

# Convert nullable Int64 columns to regular int64
int_cols_nullable = df_employee.select_dtypes(include='Int64').columns
df_employee[int_cols_nullable] = df_employee[int_cols_nullable].astype('int64')


In [ ]:
# Step: Check datatypes & memory usage of all columns
df_employee.info()

we can see a reduction from 72.5 MB to 35.3 MB. This is a good optimization method. Especially considering that in big data reducing the memory usage of the data will help with speed of all downstream tasks. 

Done for now with cleaning of Employee dataset. Analysis of data with graphs will be done after cleaning of salary dataset & merging

## Salary Dataset

### Initial Inspection

In [ ]:
import pandas as pd
from IPython.display import display

# Inspecting the salary dataset for basic information & statistics

# Shape of the dataset
print(f"Dataset shape: {df_salary.shape}")

# 2. Preview first 5 rows
display(df_salary.head())

# 3. Column names and data types
df_salary.info()

# 4. Descriptive statistics for numerical and categorical features
display(df_salary.describe(include='all'))

# 5. Check for missing values
missing_counts = df_salary.isnull().sum()
missing_perc = (missing_counts / len(df_salary) * 100).round(2)
missing_df = pd.DataFrame({'Missing Count': missing_counts, 'Missing %': missing_perc})
display(missing_df[missing_df['Missing Count'] > 0])


Issues or anomalies with dataset

- non standardized column names
- complex job_id data entry
- missing data
- improper dtypes
- Clear high value outlier in salaryInThousands column
- min value is 0

### Cleaning Salary Dataset

In [ ]:
# Rename columns
df_salary.rename(columns={
    'jobId': 'job_id',
    'salaryInThousands': 'salary_k'
}, inplace=True)

# Preview to confirm
df_salary.head(2)

In [ ]:
import pandas as pd

# Clean job_id (handle missing + strip JOB)
df_salary['job_id'] = (
    df_salary['job_id']
    .astype('string')                         # work in string mode for replace
    .str.replace('JOB', '', regex=False)      # remove 'JOB' prefix
    .astype('Int64')                          # nullable integer dtype
)

# Preview to confirm
df_salary[['job_id']].head(2)

In [ ]:
# Drop rows with missing job_id or salary_k
before_drop = len(df_salary)
df_salary = df_salary.dropna(subset=['job_id', 'salary_k'])
after_drop = len(df_salary)

print(f"Dropped {before_drop - after_drop} rows with missing job_id or salary_k")
print(f"Remaining rows: {after_drop}")


In [ ]:
# Checking column dtype & memory usage
df_salary.info()

In [ ]:
# Convert to non-nullable int64
df_salary['job_id'] = df_salary['job_id'].astype('int64')
df_salary['salary_k'] = df_salary['salary_k'].astype('int64')

print(df_salary.dtypes)

In [ ]:
# Checking column dtype & memory usage
df_salary.info()

## Merged Dataset

### Merging both datasets

In [ ]:
# Compare job_id counts and matches
emp_ids = set(df_employee['job_id'])
sal_ids = set(df_salary['job_id'])

print(f"Unique job_ids in employee dataset: {len(emp_ids)}")
print(f"Unique job_ids in salary dataset:   {len(sal_ids)}")

# Check exact match
print(f"All employee job_ids in salary dataset? {emp_ids.issubset(sal_ids)}")
print(f"All salary job_ids in employee dataset? {sal_ids.issubset(emp_ids)}")

# Find differences
missing_in_salary = emp_ids - sal_ids
missing_in_employee = sal_ids - emp_ids

print(f"Job_ids in employee but not in salary: {len(missing_in_salary)}")
print(f"Job_ids in salary but not in employee: {len(missing_in_employee)}")


Since there are missing ID's in both datasets we will use an inner join to ensure only ids that are present make it into the final merged dataset to ensure no new null values are introduced.

In [ ]:
# Track before merge
rows_emp_before = len(df_employee)
rows_sal_before = len(df_salary)

# Inner join
df_merged = df_employee.merge(df_salary, on='job_id', how='inner')

# Track after merge
rows_after = len(df_merged)

print(f"Rows in employee dataset before merge: {rows_emp_before}")
print(f"Rows in salary dataset before merge:   {rows_sal_before}")
print(f"Rows after inner join:                  {rows_after}")

print(f"Rows lost from employee dataset: {rows_emp_before - rows_after}")
print(f"Rows lost from salary dataset:   {rows_sal_before - rows_after}")


In [ ]:
# Viewing the final merged dataset to ensure inner join worked as expected
df_merged.head()

Inner join went well and minimal data was lost from the initial 1,000,000 to 999479. Now we will move onto data cleaning on merged dataset.

### Cleaning Merged dataset

In [ ]:
import pandas as pd
from IPython.display import display

# Inspecting the merged dataset for basic information & statistics

# Shape of the dataset
print(f"Dataset shape: {df_merged.shape}")

# 2. Preview first 5 rows
display(df_merged.head())

# Descriptive statistics (excluding job_id but keeping salary_k) with thousands separator
display(df_merged.drop(columns=['job_id']).describe(include='all').style.format(thousands=',', precision=2)
)

The main thing needed to be done now is to handle outliers in the numerical & categorical columns. There is also a need to check for complete duplicates.

In [ ]:
# Check for full duplicate rows
full_dupes = df_merged.duplicated().sum()
print(f"Full duplicate rows: {full_dupes}")

# Check for duplicate job_id values
jobid_dupes = df_merged['job_id'].duplicated().sum()
print(f"Duplicate job_id count: {jobid_dupes}")

# Display the first few duplicate job_id entries if present
if jobid_dupes > 0:
    display(df_merged[df_merged['job_id'].duplicated(keep=False)].sort_values('job_id'))

Lets go column by column when handling outliers

In [ ]:
viz.plot_categoricals(df=df_merged,cols=['job_role'])

From the graph, we can once again confirm, that there is a good spread of data however there is a clear singular outlier. Unfortunately, as there is no significant count of president category job_role, this means that this singular president may skew the averages of one specific industry or company or job role as he/she is not representative of the majority thus affecting eda later on.

In [ ]:
# Remove the president outlier from job_role
df_merged = df_merged[df_merged['job_role'] != 'PRESIDENT']

In [ ]:
# Remove 'PRESIDENT' from job_role if it exists
df_merged['job_role'] = df_merged['job_role'].cat.remove_unused_categories()

In [ ]:
viz.plot_categoricals(df=df_merged,cols=['education'])

This column has no major outliers, however there is a large number of NONE present however as they are explicitly labled it will not be an issue later on.

In [ ]:
viz.plot_categoricals(df=df_merged,cols=['major'])

This column has no major outliers and spread of data is actually good, however there is a large number of NONE present however as they are explicitly labled it will not be an issue later on.

In [ ]:
combined_count = df_merged['education'].isin(['HIGH_SCHOOL', 'NONE']).sum()
print(f"Combined count for HIGH_SCHOOL & NONE: {combined_count}")

There seems to be a number of missing values from the major column that are true missing values and cannot be explained by missing or semantic information from another column. This however is not an issue as we are already assuming all types of missing data to be the same and then handled as NONE.

In [ ]:
viz.plot_categoricals(df=df_merged,cols=['industry'])

This is quite unexpected. It seems the one entry we removed i.e the president row was actually the president of the country as it seems the Government column is no longer in use. Meaning that our removal of the one row was all the more valid as a country only has one president therefore he/she would not have been at all representative of the majority.

In [ ]:
# Remove 'GOVERNMENT' from job_role if it exists
df_merged['industry'] = df_merged['industry'].cat.remove_unused_categories()

In [ ]:
# Select columns for numeric distribution plot
df_numeric = df_merged[['years_experience', 'distance_from_cbd']]
viz.plot_numeric_distribution(df=df_numeric)

- years_experience column has good data spread as the boxes & whiskers are of equal length. It also has no outliers as there are no markers.

- distance_from_cbd column has generally good data spread as the whiskers and boxes are of even length. However, there are 2 high value outliers.

In [ ]:
# Inspecting distance_from_cbd outliers
df_merged[df_merged['distance_from_cbd'] > 100]

- It seems there are only 2 of these instances and are too far from 75th quanitile + 1.5 x IQR as such we will just drop these 2 rows to ensure simplicity.

In [ ]:
# Removing outliers from distance_from_cbd
df_merged = df_merged[df_merged['distance_from_cbd'] <= 100]

We will deal with salary_k next seperately. This is because there are very large value outliers.

In [ ]:
# statistics of salary_k
df_merged['salary_k'].describe()

In [ ]:
# inspecting salary_k == 0
df_merged[df_merged['salary_k'] == 0]

It clearly seems that these values were wrongly keyed in, as these people all have jobs and some even have multiple years of experience. Therefore, these rows of data are clearly invalid entries. These will be dropped.

In [ ]:
# Dropping salary_k == 0 rows
df_merged = df_merged[df_merged['salary_k'] != 0]

In [ ]:
# statistics of salary_k
df_merged['salary_k'].describe()

It also seems like there is a input of 10 million as well. Let's review the top 25 highest earners manually.

In [ ]:
# view top 25 salary_k
top_25_salaries = df_merged.nlargest(25, 'salary_k')
top_25_salaries

Clearly, we can see that the 10000000k entry is incorrect as its the only one of its kind and needs to be removed.

In [ ]:
# remove 10000000k entry
df_merged = df_merged[df_merged['salary_k'] != 10000000]

In [ ]:
# statistics of salary_k
df_merged['salary_k'].describe()

This has drastically reduced the range of values to normal human levels. Since we already reviewed the other 24 highest earners and they were genuine outliers, we cannot just haphazardly drop them. Therefore, lets verify the lowest 25 are also genuine outliers and if so let us log transform the values to reduce the skew of the data if needed.

In [ ]:
# viewing bottom 25 salary_k
bottom_25_salaries = df_merged.nsmallest(25, 'salary_k')
bottom_25_salaries

Yes this has confirmed my findings, all remaining salary_k values are valid and contain genuine outliers. Therefore, removing them is not required as they represent a portion of the job market. Lets, calculate the skew to verify if log transform is needed.

In [ ]:
print("Skewness: %f" % df_merged['salary_k'].skew())

The spread of the data is acceptable and is within the range of skewness for a fairly symmetrical bell curve distribution of between -0.5 to 0.5 

# EDA

## What is the highest paying job for the web industry?

First lets filter out dataset to only include the web industry

In [ ]:
df_web = df_merged[df_merged["industry"] == "WEB"]

Next I will group the respective jobs together and take the median value of them, we do this to get the median pay of each job. This is to ensure the average is not skewed by outliers as mean often times causes skewed data. We will also review the sample size of each job role to ensure each job role has sufficient representation.

In [ ]:
# Group by job_role in Web industry
web_roles_stats = (df_web.groupby('job_role', observed=True)['salary_k'].agg(median_salary='median', count='size').sort_values(by='median_salary', ascending=False))

display(web_roles_stats)

Here we can see that the sample size for each job role is similar therefore we can compare the jobs without any issues. Lets plot the table to a bar chart to view the data with clarity.

In [ ]:
# If you already grouped and calculated medians separately
viz.plot_bar(df=web_roles_stats.reset_index(), x_col='job_role', y_col='median_salary',title="Median Salary by Job Role in Web Industry")


From the bar chart, we can clearly see that the job role with the highest mediaan pay in the web industry is CEO.

##  Rank the top 10 jobs roles with the highest salary for all the industry?

Since this time we do not need to analyse a specific industry, we can procceed to the group by step immediately.

In [ ]:
# Group by industry + job_role and compute median salary + counts
role_salary_stats = (
    df_merged.groupby(['industry', 'job_role'], observed=True)['salary_k']
    .agg(median_salary='median', count='size')
    .reset_index()
    .sort_values(by='median_salary', ascending=False)
)

# Select the top 10 roles by median salary across all industries
top10_roles_by_industry = role_salary_stats.head(10).copy()

# Create new combined column
top10_roles_by_industry['role_label'] = (
    top10_roles_by_industry['industry'].astype(str) 
    + ' - ' + 
    top10_roles_by_industry['job_role'].astype(str)
)

# Drop the original two columns
top10_roles_by_industry = top10_roles_by_industry.drop(columns=['industry', 'job_role'])

# Reorder columns so role_label is first
cols = ['role_label'] + [c for c in top10_roles_by_industry.columns if c != 'role_label']

top10_roles_by_industry = top10_roles_by_industry[cols]

top10_roles_by_industry.reset_index(drop=True, inplace=True)

display(top10_roles_by_industry)

We can see that sample size of each job role is represented well, therefore we can proceed with the plotting of the df to make our final observation.

In [ ]:
viz.plot_bar(df=top10_roles_by_industry.reset_index(), x_col='role_label', y_col='median_salary',title="Top 10 Job Roles across all industries by Median Salary")

These are the top 10 jobs ranked by median salary across all industries with the highest paid job roles on the left and the lowest on the right side.

## Which of the industries has the highest salary?

This time since we are only comparing across industries which contains a range of jobs, we need to have a robust method to compare the average salary of each industry. So what I will do is compare is highest lowest and median salary across all industries to get a more overarching view of the salaries of each type of individual i.e. median -> Typical employee, lowest -> lowest paying job-role & the highest -> best paying job-role in the industry. This should help the government understand which industries allow for growth of the employees and which industries have stagnant pay.

In [ ]:
# For each industry, get lowest, median, and highest job-role medians
industry_salary_profile = (
    df_merged.groupby('industry', observed=True)['salary_k']
    .agg(count='size', lowest_salary='min', median_salary='median', highest_salary='max')
    .reset_index()
    .sort_values(by='median_salary', ascending=False)
)

In [ ]:
# Calculate delta (growth potential)
industry_salary_profile['salary_delta'] = industry_salary_profile['highest_salary'] - industry_salary_profile['lowest_salary']
industry_salary_profile.reset_index(drop=True, inplace=True)

# Display the industry salary profile
display(industry_salary_profile)

Here we can see that the sample size for each of the industries are very similar therefore we can compare without any additional steps.

In [ ]:
viz.plot_clustered_bars(
    df=industry_salary_profile,
    x_col='industry',
    y_cols=['median_salary', 'highest_salary', 'lowest_salary','salary_delta'],
    title="Median Salary, Highest Salary & Salary Delta by Industry"
)

This clustered bar chart tells us quite a lot of information. We can observe that the typical employee in the Oil & Finance industry get paid the most at 128k. However, to differenciate between the two industries to find the "highest" paying we can then compare the highest pay, lowest pay & salary delta. Between the two industries, the oil industry has the highest salary at 301k as compared to the finance industries highest salary of 294k. Furthermore, when comparing the lowest salary, we can see that there is only a small difference of 1k but the oil industry is better paying at 37k as compared to the finance industry's 36k. When comparing the salary delta (Potential for growth) we can see that thee oil industry comes out ahead again at 264k compared to the finance industries 258k. Therefore, in conclusion typically for most employees, either the finance or oil industry will lead to the highest pay. However, if selecting the highest paying industry, the oil industry is the best paying industry when comparing all different levels of pay as well as the potential for income growth.

## Which job has the lowest pay?